# [IAPR 2020:][iapr2020] Project ‒ Special Project¶
**Author:** Quentin Talon & Albéric de Lajarte  
**Due date:** 29.05.2020  
[iapr2018]: https://github.com/LTS5/iapr-2020

## Extract datas

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pims

data_base_path = os.path.join(os.pardir, 'data')
vid = pims.open(os.path.join(data_base_path, 'robot_parcours_1.avi'))
plt.figure(figsize = (10,10))
plt.imshow(vid[0])
print(vid)

## Part 1: Still and moving image segmentation

### Background
We simply calculate the mean along all the images. This reduces the noise and removes the moving object.

In [ ]:
vid_stack = np.stack(vid, axis=0)
print("Shape of vid_stack:{}".format(vid_stack.shape))
background = np.median(vid_stack, axis=0).astype(int)
print("Shape of background:{}".format(background.shape))
plt.figure(figsize = (20,20))
plt.imshow(background)
plt.show()

### Moving part
This is simply a difference of image and the backgound